In [1]:
import os
from PIL import Image
from transformers import PaliGemmaProcessor, PaliGemmaForConditionalGeneration
from transformers.image_utils import load_image
import torch
import gc
import traceback
import numpy as np
import PIL.Image


/home/elboardy/.local/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/elboardy/.local/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/elboardy/.local/lib/python3.9/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the A

In [2]:
cases_dir = '/media/RLAB-Disk01/(final)merged_images_with_labels_order_and_folders_mask_normalized/'
cache_dir = '/media/RLAB-Disk01/Large-Language-Models-Weights'
model_id = "google/paligemma2-28b-pt-896"
offload_folder = '/media/RLAB-Disk01/Large-Language-Models-Weights'
    

In [3]:
processor = PaliGemmaProcessor.from_pretrained(model_id, 
                                               cache_dir=cache_dir,
                                               #local_files_only=True
                                               )
model = PaliGemmaForConditionalGeneration.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        cache_dir=cache_dir,
        attn_implementation="sdpa",
        offload_folder=offload_folder,
        # local_files_only=True
    ).eval()


Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

In [4]:
cases_to_process = []
for case in os.listdir(cases_dir):
    case_dir = os.path.join(cases_dir, case)
    if os.path.isdir(case_dir):
        response_path = os.path.join(case_dir, 'paligemma2-28b-report.txt')
        if not os.path.exists(response_path):
            cases_to_process.append(case)
print(f"Found {len(cases_to_process)} cases to process.")

Found 39 cases to process.


In [5]:


def read_image_local(path):
    # Open image from local path
    image = PIL.Image.open(path)
    # Convert to numpy array
    image = np.array(image)
    # Remove alpha channel if necessary
    if image.ndim == 3 and image.shape[2] == 4:
        image = image[:, :, :3]
    return image




failed_cases = []
    
for case in cases_to_process:
        case_dir = os.path.join(cases_dir, case)
        try:
            # Load clinical info
            clinical_path = os.path.join(case_dir, 'diagnostic_prompt.txt')
            if not os.path.exists(clinical_path):
                print(f"Missing clinical info for {case}")
                failed_cases.append(case)
                continue
                
            clinical_info = open(clinical_path).read()

            # Build prompts
            system_prompt = """<image> \nConsider that you are a professional radiologist with several years of experience and you are now treating a patient. Write a fully detailed diagnosis report for this case, avoiding any potential hallucination and paying close attention to all of the batch images attached to this message.

Use the following structure for the report:

## Radiologist's Report

### Patient Information:
- *Age:* 65
- *Sex:* Male
- *Days from earliest imaging to surgery:* 1
- *Histopathological Subtype:* Glioblastoma
- *WHO Grade:* 4
- *IDH Status:* Mutant
- *Preoperative KPS:* 80
- *Preoperative Contrast-Enhancing Tumor Volume (cm³):* 103.21
- *Preoperative T2/FLAIR Abnormality (cm³):* 36.29
- *Extent of Resection (EOR %):* 100.0
- *EOR Type:* Gross Total Resection (GTR)
- *Adjuvant Therapy:* Radiotherapy (RT) + Temozolomide (TMZ)
- *Progression-Free Survival (PFS) Days:* 649
- *Overall Survival (OS) Days:* 736

#### Tumor Characteristics:

#### Segmentation Analysis:

#### Surgical Considerations:

### Clinical Summary:

### Recommendations:

### Prognostic Considerations:

### Follow-Up Plan:

### Additional Notes*(if any)*:

Ensure all findings from all of the images and clinical data provided. Please mention at the end of the report how many images were reviewed."""

            user_prompt = f"""You will be given batches of images, which are different sequences of MRI scans. 
    The images are for patients who are likely to have a brain tumor. Each image will contain up to 10 slices for 5 different sequences and the segmentation masks for the tumor at the bottom row of the image. 
    Additional clinical data about the patient is: 
    {clinical_info}"""


            full_prompt = system_prompt + "\n\n" + user_prompt

            print(f"Processing {case}")
            # Load image
            image_files = [f for f in os.listdir(case_dir) if f.lower().endswith('.png')]
            if not image_files:
                print(f"No images found for {case}")
                failed_cases.append(case)
                continue
                
            image_path = os.path.join(case_dir, image_files[-1])  # Using last image
            image = read_image_local(image_path)
            print(f"Loaded image: {image_path}")
        
            # Process inputs
            model_inputs = processor(
                text=full_prompt,
                images=image,
                return_tensors="pt"
            ).to(model.device)


            model_inputs = model_inputs.to(dtype=model.dtype)
            input_len = model_inputs["input_ids"].shape[-1]

            print("Generated inputs")
            # Generate response
            with torch.inference_mode():
                generation = model.generate(
                    **model_inputs,
                    max_new_tokens=4096,
                    do_sample=False,
                    temperature=0.7,
                    top_p=0.9,
                )
            print("Generated response")
            print(generation)    
            generation = generation[0][input_len:]
            response = processor.decode(generation, skip_special_tokens=True)
            print(f"Generated response: {response}")
            # Save response
            with open(os.path.join(case_dir, 'paligemma2-28b-report.txt'), 'w', encoding='utf-8') as f:
                f.write(response.strip())

            # Cleanup
            del model_inputs, generation, response
            gc.collect()
            torch.cuda.empty_cache()

        except Exception as e:
            error_msg = f"\nError processing {case}:\n{traceback.format_exc()}"
            print(error_msg)
            failed_cases.append(case)

    # Save failed cases
if failed_cases:
        with open(os.path.join(cases_dir, 'failed_paligemma2-28b.txt'), 'w') as f:
            f.write("\n".join(failed_cases))
            

/home/elboardy/anaconda3/envs/paligemma2/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/elboardy/anaconda3/envs/paligemma2/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


Processing RHUH-0001
Loaded image: /media/RLAB-Disk01/(final)merged_images_with_labels_order_and_folders_mask_normalized/RHUH-0001/RHUH-0001_batch_6.png
Generated inputs
Generated response
tensor([[257152, 257152, 257152,  ..., 235279, 235274,      1]],
       device='cuda:0')
Generated response: T1
Processing RHUH-0002
Loaded image: /media/RLAB-Disk01/(final)merged_images_with_labels_order_and_folders_mask_normalized/RHUH-0002/RHUH-0002_batch_6.png
Generated inputs
Generated response
tensor([[257152, 257152, 257152,  ...,    108, 235290,      1]],
       device='cuda:0')
Generated response: -
Processing RHUH-0003
Loaded image: /media/RLAB-Disk01/(final)merged_images_with_labels_order_and_folders_mask_normalized/RHUH-0003/RHUH-0003_batch_6.png
Generated inputs
Generated response
tensor([[257152, 257152, 257152,  ...,    108, 235318,      1]],
       device='cuda:0')
Generated response:     PFS: 0
6
Processing RHUH-0004
Loaded image: /media/RLAB-Disk01/(final)merged_images_with_labels_o